In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

In [2]:
# Registers
dq = QuantumRegister(7, "data")
aq = QuantumRegister(3, "anc")
mreg = ClassicalRegister(3, "m")
dreg = ClassicalRegister(7, "d")

qc = QuantumCircuit(dq, aq, mreg, dreg)

K_supports = {
    0: [3, 4, 5, 6],
    1: [0, 2, 4, 6],
    2: [1, 2, 5, 6],
}

In [3]:
# Stabilizer measurements
for a_idx in range(3):
    anc = aq[a_idx]
    qc.h(anc)
    for t in K_supports[a_idx]:
        qc.cx(anc, dq[t])
    qc.h(anc)
    qc.measure(anc, mreg[a_idx])

# Conditional corrections
for val in range(1, 8):
    sub_q = QuantumRegister(7, "sub")
    blk = QuantumCircuit(sub_q)
    blk.z(sub_q[val - 1])
    qc.if_test((mreg, val), blk, qubits=dq, clbits=[])

qc.barrier()
qc.measure(dq, dreg)

In [4]:
fig = qc.draw(output="mpl", scale=0.5)   # matplotlib Figure
plt.tight_layout()
plt.show()

In [5]:
# --- THE IMPORTANT FIX ---
sim = AerSimulator()
result = sim.run(qc, shots=5000).result()   # NO transpile(qc, sim)

counts = result.get_counts() # stores dreg + mreg -> need to get only dreg counts

In [6]:
counts_dreg = {}
for key, value in counts.items():
    dreg_key = key[:7]  # first 7 bits correspond to dreg
    if dreg_key in counts_dreg:
        counts_dreg[dreg_key] += value
    else:
        counts_dreg[dreg_key] = value

print(counts_dreg)

{'1100110': 633, '0000000': 611, '1111000': 625, '1001011': 620, '0110011': 611, '1010101': 598, '0101101': 609, '0011110': 693}


In [7]:
# plot histogram for dreg counts

plot_histogram(counts_dreg)
plt.tight_layout()
plt.show()